In [47]:
import random
import re
import time
from datetime import datetime, timedelta
from functools import partial

import numpy as np
import pandas as pd
import torch
from sklearn.utils import shuffle
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, random_split, RandomSampler
from torch.optim.adamw import AdamW
from transformers import BertForSequenceClassification, BertConfig, AutoTokenizer, AutoModel
from transformers import get_linear_schedule_with_warmup


from ml.src.data.load_datasets import load_RuTweetCorp, DATA_PATH

In [6]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%autoreload 2

In [8]:
data_path = load_RuTweetCorp(concat=True)

ParserError: Error tokenizing data. C error: EOF inside string starting at row 109353

In [9]:
# data_path = DATA_PATH / 'raw' / 'ru-tweet-corp.csv'

In [10]:
data = pd.read_csv(data_path, header=None)

C:\Users\uiqko\AppData\Local\Temp\ipykernel_5500\2784446461.py:1: DtypeWarning: Columns (3,4,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_path, header=None)


In [11]:
data = shuffle(data)

In [12]:
data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
207684,92774,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,...,NaN,4.222319e+17,1.389503e+09,darnellhphedgep,я хочу тебя найти....но не знаю где искать(,-1.0,3637.0,267.0,35.0,0.0
41435,41435,4.099313e+17,1.386570e+09,xizydinivem,З новым годам родны край. I другiя края таксам...,1.0,0,0,0,531.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35198,35198,4.097734e+17,1.386533e+09,mihail_za_73034,"RT @anton_la_752289: кто то написал: ""ЛЮБИЛА е...",1.0,0,1,0,872.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59996,59996,4.101440e+17,1.386621e+09,zellenkov,@gasparcompany @xenia_sobchak воронины епть! К...,1.0,0,0,0,779.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179344,64434,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,...,NaN,4.169686e+17,1.388248e+09,blueberries_kap,"@FatStupidDeer я ушел с дискоча нафиг, я еще т...",-1.0,6286.0,787.0,455.0,13.0
180756,65846,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,...,NaN,4.172103e+17,1.388306e+09,KateMaskevich,расстроена=( такое западло да под Новый Год!,-1.0,17.0,4.0,5.0,0.0
15785,15785,4.093530e+17,1.386432e+09,ZzzCalifornia,интересно после грибов тольк мне весело?не зря...,1.0,0,0,0,2398.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197204,82294,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,...,NaN,4.198281e+17,1.388930e+09,KseniaIvantsova,"@DMC_SERGHO @Club_Relax_ Вот они эти мишки, ко...",-1.0,4632.0,69.0,31.0,2.0
202626,87716,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,...,NaN,4.212637e+17,1.389272e+09,lutikova_li,@fedechka_m кстати что-то я нигде не смогла на...,-1.0,691.0,18.0,23.0,0.0
2531,2531,4.089336e+17,1.386332e+09,JuliAussie,RT @Dance_with_me_: Кто бы знал как я не люблю...,1.0,0,1,0,36948.0,...,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
X, y = data.iloc[:, 4], data.iloc[:, 5]

In [14]:
X.dropna(inplace=True)
y.dropna(inplace=True)

In [15]:
X.head(10)

41435    З новым годам родны край. I другiя края таксам...
35198    RT @anton_la_752289: кто то написал: "ЛЮБИЛА е...
59996    @gasparcompany @xenia_sobchak воронины епть! К...
15785    интересно после грибов тольк мне весело?не зря...
2531     RT @Dance_with_me_: Кто бы знал как я не люблю...
43926    Завтра последний рывок иии.. Свободааа\r\nПрав...
36726    У моей маленькой 3-родной сестренки, которой н...
97400    Сижу на олимпиаде по физической культуре)\r\nН...
35467    четкость этого фото просто зашкаливает:DDD htt...
89994    Он должен быть крутым, я уверен!:-D http://t.c...
Name: 4, dtype: object

In [16]:
y.head(10)

41435    1.0
35198    1.0
59996    1.0
15785    1.0
2531     1.0
43926    1.0
36726    1.0
97400    1.0
35467    1.0
89994    1.0
Name: 5, dtype: float64

In [17]:
clean_tweet = partial(re.sub, '@\w+')

In [44]:
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased', one_hot_labels=True)

In [45]:
X[0]

'Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D'

In [46]:
tokenizer.tokenize(X[0])

['Да',
 ',',
 'все',
 '-',
 'таки',
 'он',
 'немного',
 'похож',
 'на',
 'него',
 '.',
 'Но',
 'мой',
 'мальчик',
 'все',
 'равно',
 'лучше',
 ':',
 'D']

In [21]:
tokenizer.convert_tokens_to_ids(_)

[14912,
 128,
 4752,
 130,
 24541,
 2886,
 17042,
 15833,
 1469,
 7268,
 132,
 8312,
 28635,
 28677,
 4752,
 17561,
 17874,
 156,
 176]

In [22]:
X.map(len).mean()

85.43346967191715

In [23]:

input_ids = []
attention_masks = []

for x in X:
    encoded = tokenizer.encode_plus(
        x,
        add_special_tokens=True,
        max_length=100,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])


In [24]:
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

In [25]:
input_ids

tensor([[  101,   791, 15485,  ...,     0,     0,     0],
        [  101, 41286,   168,  ...,     0,     0,     0],
        [  101,   168, 12399,  ...,     0,     0,     0],
        ...,
        [  101,   168, 31106,  ...,     0,     0,     0],
        [  101,   168, 59726,  ...,     0,     0,     0],
        [  101,   168, 20954,  ...,     0,     0,     0]])

In [26]:
labels = torch.tensor(y)

In [27]:
X[0]

'Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D'

In [28]:
input_ids[0]

tensor([  101,   791, 15485, 16170,  3687,  1468, 11172,   132,   186,  3487,
          249,   878, 11744, 37831, 10338,   855, 15485, 16170,   122,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [29]:
labels[0]

tensor(1., dtype=torch.float64)

In [30]:
dataset = TensorDataset(input_ids, attention_masks, labels)

In [31]:
train_size = int(len(dataset) * 0.9)
val_size = len(dataset) - train_size

In [32]:
train_dataset, val_dataset = random_split(dataset, (train_size, val_size))

In [33]:
batch_size = 32

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=batch_size
)

val_dataloader = DataLoader(
    val_dataset,
    sampler=RandomSampler(val_dataset),
    batch_size=batch_size
)

In [34]:
model = AutoModel.from_pretrained(
    'DeepPavlov/rubert-base-cased',
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 199 different named parameters.

==== Embedding Layer ====

embeddings.word_embeddings.weight                       (119547, 768)
embeddings.position_embeddings.weight                     (512, 768)
embeddings.token_type_embeddings.weight                     (2, 768)
embeddings.LayerNorm.weight                                   (768,)
embeddings.LayerNorm.bias                                     (768,)

==== First Transformer ====

encoder.layer.0.attention.self.query.weight               (768, 768)
encoder.layer.0.attention.self.query.bias                     (768,)
encoder.layer.0.attention.self.key.weight                 (768, 768)
encoder.layer.0.attention.self.key.bias                       (768,)
encoder.layer.0.attention.self.value.weight               (768, 768)
encoder.layer.0.attention.self.value.bias                     (768,)
encoder.layer.0.attention.output.dense.weight             (768, 768)
encoder.layer.0.attention.output.dense.bias                   

In [36]:
optimizer = AdamW(model.parameters())

In [37]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round(elapsed))

    # Format as hh:mm:ss
    return str(timedelta(seconds=elapsed_rounded))

In [38]:
from sklearn.metrics import accuracy_score

In [39]:
seed_val = 42

random.seed(42)
np.random.seed(seed_val)
torch.manual_seed(seed_val)

In [40]:
epochs = 10
train_stats = []


In [41]:
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0, # Default value in run_glue.py
    num_training_steps=len(dataset) * epochs
)

In [42]:
device = torch.device('cpu')

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

In [43]:
total_t0 = time.time()
correct = 0

for epoch in range(1, epochs + 1):
    print(f'Epoch {epoch} / {epochs}')
    print('Training...')

    t0 = time.time()
    total_train_loss = 0

    model.train()

    for i, batch in enumerate(train_dataloader):
        model.zero_grad()
        ids, mask, labels_ = batch

        ids = ids.to(device)
        mask = mask.to(device)
        labels_ = labels_.to(device)

        outputs = model(
            ids,
            token_type_ids=None,
            attention_mask=mask,
        )

        pred = torch.argmax(outputs.logits, dim=1)
        loss = loss_fn(pred, labels_)

        total_train_loss += loss.item()
        correct += torch.sum(pred == labels_)
        
        loss.backward()

        torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = np.average(total_train_loss)

    train_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(train_time))

    # validation
    print('validation...')

    t0 = time.time()
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    eval_steps = 0

    for batch in val_dataloader:
        ids, mask, labels_ = batch
        print(ids.size, mask.size, labels.size)
        ids = ids.to(device)
        mask = mask.to(device)
        labels_ = labels_.to(device)

        with torch.no_grad():
            outputs = model(
                ids,
                token_type_ids=None,
                attention_mask=mask,
            )

            pred = torch.argmax(outputs.logits, dim=1)
            loss = loss_fn(pred, labels_)

        total_eval_loss += loss.item()
        # logits = logits.
        total_eval_accuracy += accuracy_score(logits, labels)

    avg_val_accuracy = np.average(total_eval_accuracy)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = np.average(total_eval_loss)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    train_stats.append({
        'epoch': epoch,
        'train loss': avg_train_loss,
        'valid loss': avg_val_loss,
        'valid accuracy': avg_val_accuracy,
        'train time': train_time,
        'validation time': validation_time,
    })

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

Epoch 1 / 10
Training...


TypeError: forward() got an unexpected keyword argument 'labels'

In [ ]:
labels.size()